In [162]:
#from xvfbwrapper import Xvfb

#vdisplay = Xvfb()
#vdisplay.start()

# for environ
import os

# only using device 0
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
#os.environ["LIBGL_ALWAYS_SOFTWARE"]="1"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

# importing tensorflow
import tensorflow as tf

# import scipy
import scipy, csv

import pandas as pd

import tensorflow_probability as tfp

import cvxpy as cp

In [2]:
import gym
from gym import wrappers
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm

# to display environment data
# see https://gist.github.com/thomelane/79e97630ba46c45985a946cae4805885
from IPython import display

In [3]:
import sys
sys.path += ['/localhome/volodin/local/usr/bin']
sys.path

['',
 '/localhome/volodin/miniconda3/envs/neuronfailure/lib/python36.zip',
 '/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6',
 '/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6/lib-dynload',
 '/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6/site-packages',
 '/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6/site-packages/IPython/extensions',
 '/localhome/volodin/.ipython',
 '/localhome/volodin/local/usr/bin']

In [4]:
from xvfbwrapper import Xvfb

vdisplay = Xvfb()
vdisplay.start()

In [3]:
# https://www.cvxpy.org/

# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A*x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

[-1.79109253e-19  2.85112420e-02  2.79973443e-19  3.37658751e-20
 -2.72802659e-19  1.49285011e-01 -9.97212062e-20  8.35373892e-20
  2.46718649e-01  5.78224144e-01 -4.03739462e-19  1.01242860e-03
 -9.28486200e-20  2.26767464e-01 -1.58813677e-19 -8.97232308e-20
 -1.22145726e-19 -1.51509432e-19  1.12060673e-19 -3.48318630e-19]
[ 2.50938945  0.          2.78354615  1.79425782 13.08579183  0.
  0.73716363  3.35344995  0.          0.          8.93825054  0.
  7.02955161  0.          4.71068649  3.18873635  2.06090107 10.08166738
  3.0481157   8.53268239]


In [4]:
tf.reset_default_graph()
# allowing GPU memory growth to allocate only what we need
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
sess = tf.Session(config=config, graph = tf.get_default_graph())

In [5]:
env = gym.make('CartPole-v0')
#env = wrappers.Monitor(env, 'video')
env.reset()

array([ 0.03118045, -0.0445104 , -0.02628692, -0.04654129])

In [6]:
env.action_space

Discrete(2)

In [7]:
S_DIM = 4
ACTIONS = 2

In [8]:
# states
states = tf.placeholder(tf.float64, shape = (None, S_DIM,))

# taken actions
actions = tf.placeholder(tf.int64, shape = (None,))

# rewards obtained
rewards = tf.placeholder(tf.float64, shape = (None,))

# costs obtained
costs = tf.placeholder(tf.float64, shape = (None,))

# discounting factor
gamma = tf.placeholder(tf.float64, shape = ())

def fc_layer(x, n, activation = tf.nn.sigmoid):
    """ Fully connected layer for input x and output dim n """
    return tf.contrib.layers.fully_connected(x, n, activation_fn=activation,
    weights_initializer=tf.initializers.lecun_normal(), weights_regularizer=None,
    biases_initializer=tf.zeros_initializer(), biases_regularizer=None, trainable=True)

In [9]:
# layers
with tf.name_scope('layers'):
    z = states # state is an input
    z = fc_layer(z, 10)
    z# = fc_layer(z, 10)
    #z = fc_layer(z, 10)
    z = fc_layer(z, ACTIONS, activation = None)
    output = z

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# softmax to make probability distribution
logits = tf.nn.softmax(output)

# predicted labels
labels = tf.argmax(logits, axis = 1)

In [11]:
params = tf.trainable_variables()

Want to optimize the sum of rewards:
$$
J(\theta)=\mathbb{E}_{\pi(\theta)}\sum\limits_{t=0}^\infty r_t
$$

Take the gradient and use log-likelihood trick:
$$
\nabla J(\theta)=\mathbb{E}_{\pi}\sum\limits_{t=0}^\infty r_t\nabla_\theta \log \pi(a_t|s_t)=\nabla_\theta\mathbb{E}_{\pi}\sum\limits_{t=0}^\infty r_t\log \pi(a_t|s_t)
$$

Therefore the task is equivalent to minimizing a loss of
$$
\mathbb{E}_{\pi}\sum\limits_{t=0}^\infty r_t\log \pi(a_t|s_t)
$$

Which is estimated stochastically using episodes.

## Safe algo 1: <a href="https://arxiv.org/pdf/1705.10528.pdf">Constrained Policy Optimization</a>

Estimating $g=\nabla_\theta J_r(\pi)$ using policy gradients

Estimating $b=\nabla_\theta J_C(\pi)$ using policy gradients

Estimating $H=\mathbb{E}\frac{\partial^2}{\partial \theta^2}D_{KL}(\theta'|\theta)$

Constants: $\delta$ step size for trust region, $c=J_C-d$, where $d$ is the maximal per-episode value.

In [12]:
def dz_dw_flatten(z, params):
    """ Calculate dz/dparams and flatten the result """
    return tf.concat([tf.reshape(x, shape = (-1,)) for x in tf.gradients(z, params)], axis = 0)

def iterate_flatten(tensor):
    """ Iterate over flattened items of a tensor """
    if type(tensor) == list:
        for t in tensor:
            for v in iterate_flatten(t):
                yield v
    elif len(tensor.shape) == 0:
        yield tensor
    else:
        for idx in range(tensor.shape[0]):
            for v in iterate_flatten(tensor[idx]):
                yield v
                
def tf_hessian(var, params):
    # gradients of the loss w.r.t. params
    grads = tf.gradients(var, params)
    grad_components = list(iterate_flatten(grads))
    hessian = [dz_dw_flatten(t, params) for t in tqdm(grad_components)]
    return hessian

In [13]:
# one-hot encoded actions
a_one_hot = tf.one_hot(actions, ACTIONS)

# taken logits
#logits_taken = tf.gather(logits, actions, axis = 1)
logits_taken = tf.boolean_mask(logits, a_one_hot)

# logarithm
log_logits = tf.log(logits_taken)

# logarithm of logits * reward
r_log_logits_sum = tf.reduce_sum(tf.multiply(rewards, log_logits))

# cost return
J_c = tf.reduce_sum(costs)

# calculated loss
loss_r = -tf.reduce_sum(tf.multiply(log_logits, tf.cumsum(rewards, reverse = True)))
loss_c = -tf.reduce_sum(tf.multiply(log_logits, tf.cumsum(costs, reverse = True)))

# KL(pi(a)||pi(a) fixed)
kl_div_var_fixed = tf.reduce_mean(tfp.distributions.kl_divergence(
    tf.distributions.Categorical(probs = logits),
    tf.distributions.Categorical(probs = tf.stop_gradient(logits))))
    #tf.distributions.Categorical(probs = logits))) #!!! not using this because want variable first parameter and fixed second

# policy gradient for reward
g = list(iterate_flatten(tf.gradients(-loss_r, params)))

# policy gradient for constraint
b = list(iterate_flatten(tf.gradients(-loss_c, params)))

# hessian of KL divergence (parameter H)
H = tf_hessian(kl_div_var_fixed, params)

# flattened parameters
theta = list(iterate_flatten(params))

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use tf.cast instead.


/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
100%|██████████| 72/72 [00:10<00:00,  6.04it/s]


In [14]:
def sample_action(observation):
    """ Sample an action from the policy """
    
    if np.random.random() <= eps:
        return env.action_space.sample()
    
    p = sess.run(logits, feed_dict = {states: [observation]})[0]
    return np.random.choice(range(2), p = p)

In [15]:
observations = []

In [16]:
def cost(obs):
    """ Calculate scalar cost of one observation """
    assert isinstance(obs, np.ndarray) and obs.shape == (4,), "Input must be an np-array [x xdot phi phidot]"
    
    # parsing input
    x, x_dot, phi, phi_dot = obs
    
    X_MAX = 1.0
    X_DOT_MAX = 0.5
    PHI_MAX = 0.1
    PHI_DOT_MAX = 0.5
    
    if np.any(np.abs([x, x_dot, phi, phi_dot]) > [X_MAX, X_DOT_MAX, PHI_MAX, PHI_DOT_MAX]):
        return 1
    
    # in all other cases no cost
    return 0

In [17]:
def get_rollout():
    """ Obtain rollout using policy """
    done = False
    observation = env.reset()
    sarc = []
    while not done:
        act = sample_action(observation)
        observation_, reward, done, info = env.step(act) # take a random action
        cost_ = cost(observation_)
        global observations
        observations += [observation_]
        sarc.append((observation, act, reward, cost_))
        observation = observation_
    env.close()
    return sarc

In [18]:
class OwnGradientDescent():
    def __init__(self, gamma = 0.5, theta = 0.9):
        # gamma (learning rate)
        self.gamma = tf.Variable(gamma, dtype = tf.float32)
        self.theta = theta
        
    def minimize(self, loss, params):
        """ Minimize some loss """
        def decrement_weights(W, gamma, grads):
            """ w = w - how_much """
            ops = [w.assign(tf.subtract(w, tf.multiply(gamma, grad))) for w, grad in zip(W, grads)]
            return tf.group(ops)
        
        # gradients of the loss w.r.t. params
        grads = tf.gradients(loss, params)
        
        # perform gradient descent step
        train_op = decrement_weights(params, self.gamma, grads)
        
        # updating gamma
        upd_op = self.gamma.assign(tf.multiply(self.gamma, self.theta))
        
        return tf.group(train_op, upd_op)

In [19]:
opt = tf.train.AdamOptimizer(0.005).minimize(loss_r)

/localhome/volodin/miniconda3/envs/neuronfailure/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [20]:
# one learning iteration
#step = learning_step()
step = opt

In [21]:
def train_step(gamma_):
    # obtaining rollout data
    S, A, R, C = list(zip(*get_rollout()))
    sess.run(step, feed_dict = {states: S, actions: A, rewards: R, costs: C, gamma: gamma_})
    return np.sum(R)

In [22]:
init = tf.global_variables_initializer()
sess.run(init)
r = []

In [23]:
# non-greediness
eps = 0.1

In [24]:
for i in tqdm(range(10)):
    #gamma_ = (1000 - i) / 50000.
    gamma_ = 0.05
    r += [train_step(gamma_)]

100%|██████████| 10/10 [00:00<00:00, 26.87it/s]


In [25]:
len(observations)

288

In [238]:
def solve_CPO(S, A, R, C, delta = 0.1, d = 50, H_lambda = 0.01):
    # CPO hyperparameters

    # D_KL maximal distance
    #delta

    # maximal constraint return
    #d

    # matrix conditioner
    #H_lambda
    
    # calculating H, g, b, J from obtained data
    H_, J_c_, b_, g_ = sess.run([H, J_c, b, g], feed_dict = {states: S, actions: A, rewards: R, costs: C})
    
    # converting to matrix form
    g_ = np.array([g_]).T
    b_ = np.array([b_]).T
    H_ = np.array(H_) + np.eye(len(H_)) * H_lambda

    # constraint dissatisfaction
    c = J_c_ - d

    # current parameters
    theta_k = list(iterate_flatten(sess.run(params)))
    
    # https://www.cvxpy.org/

    # Construct the problem.
    theta = cp.Variable(72)
    objective = cp.Maximize(g_.T @ (theta - theta_k))
    constraints = [c + b_.T @ (theta - theta_k) <= 0, cp.quad_form(theta - theta_k, H_) <= 2 * delta]
    prob = cp.Problem(objective, constraints)

    # The optimal objective value is returned by `prob.solve()`.
    result = prob.solve(gp=False)
    # The optimal value for x is stored in `x.value`.
    print(theta.value)
    # The optimal Lagrange multiplier for a constraint is stored in
    # `constraint.dual_value`.
    #print(constraints[0].dual_value, constraints[1].dual_value)
    
    load_params(params, theta.value)

In [ ]:
def load_params(params, vector):
    """ Params: list of tensors, vector: array with new values (shape as flattened params) """
    # index to start
    idx = 0
    
    # array with assign operators
    assigns = []
    
    # loop over parameters
    for p in params:
        # length of current parameter (flattened)
        curr_len = np.prod(p.shape)
        
        # new values for p
        p_new_value = vector[idx:idx+curr_len]
        
        # reshaping the values
        assigns.append(tf.assign(p, p_new_value.reshape(p.shape)))
        
        # incrementing start ID
        idx += curr_len
        
    # all assigns in one operation
    assigns = tf.group(assigns)
    
    # running the operation
    sess.run(assigns)

In [269]:
S, A, R, C = zip(*get_rollout())
solve_CPO(S, A, R, C, delta = 0.1, d = 50, H_lambda = 0.01)

[-0.5260213  -0.10875747  0.35197788  0.26378278  0.40846158  0.31948562
  1.43602992 -0.51994258  0.03880309  0.80911927 -0.19664552 -0.58506983
 -0.33681947 -0.63787194 -0.03463686 -0.73208875 -0.43207586  0.28735122
 -0.93439039 -0.35168522 -0.49272021 -0.51425619  0.3003644   0.50199689
 -1.35361282  0.34054148 -0.91492341 -0.7878181  -0.22093869 -0.55555895
  0.54767889  0.01578584  0.66748887  0.01259661 -0.30303579  0.05088231
 -0.40347277 -0.14404383  0.09403651 -0.48966935 -2.83779657 -0.86497546
 -2.1253072  -3.02681871  3.51305572 -3.06627353  4.20826376  2.90912974
 -2.17443829  3.93894661  0.9765633  -1.46515016 -0.29649485 -0.23517808
  1.18946243 -0.92512621  2.02227105 -1.66429667 -3.93213645  3.13152796
  1.76809673 -1.55149009 -3.99923354  4.25533037 -2.2847743   2.9485421
  0.86472045 -0.89832846 -4.46043278  3.62145951 -2.84836801  2.82856041]


In [270]:
np.sum(R)

10.0

In [125]:
# https://www.cvxpy.org/

# Construct the DUAL problem (doesn't work...)
nu = cp.Variable(1)
lambda_ = cp.Variable(1)
objective = cp.Maximize(-(gHg - 2 * r_ * nu + nu ** 2 * S_) + nu * c - lambda_ * delta / 2)
constraints = [0 <= nu, 0 <= lambda_]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve(gp=False)
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value, constraints[1].dual_value)

DCPError: Problem does not follow DCP rules. However, the problem does follow DGP rules. Consider calling this function with `gp=True`.

In [88]:
env = env.unwrapped

In [ ]:
env = gym.wrappers.Monitor(env, './video')
get_rollout()

In [47]:
tf.train.Saver().save(sess, './cartpole-h1.ckpt')

'./cartpole-h1.ckpt'

In [23]:
tf.train.Saver().restore(sess, './cartpole-h1.ckpt')

INFO:tensorflow:Restoring parameters from ./cartpole-h1.ckpt
